In [29]:
processed_text = []
for s in text:
    s = s.split()
    processed_text.append(s)

processed_text

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'u', 'oni'],
 ['free',
  'entry',
  'in',
  '2',
  'a',
  'wkly',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  '21st',
  'may',
  '2005',
  'text',
  'fa',
  'to',
  '87121',
  'to',
  'receive',
  'entry',
  'questionstd',
  'txt',
  "ratetc's",
  'apply',
  "08452810075over18's"],
 ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say'],
 ['nah',
  'i',
  "don't",
  'think',
  'he',
  'goes',
  'to',
  'usf',
  'he',
  'lives',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darling',
  "it's",
  'been',
  '3',
  "week's",
  'now',
  'and',
  'no',
  'word',
  'back',
  "i'd",
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',
  'still?',
  'tb',
  'ok',
  'xxx',
  'std',
  

In [30]:
vocab = set()
for s in processed_text:
    for word in s:
        vocab.add(word)
    
len(vocab)

10674

In [32]:
X = np.zeros((len(processed_text), len(vocab)))

for i, s in enumerate(processed_text):
    for word in s:
        X[i, list(vocab).index(word)] += 1
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [44]:
y = np.zeros((len(processed_text), 1))

for i, l in enumerate(label):
    if l == 'ham':
        y[i] = 0
    else:
        y[i] = 1
y

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5014, 10674)
(558, 10674)
(5014, 1)
(558, 1)


In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [47]:
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_train, y_train)


c:\Users\HayXanhMan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9970083765456721

In [48]:
model.score(X_test, y_test)

0.982078853046595

In [63]:
new_message = "go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat..."

In [68]:
new_message = re.sub('[.,*!@#$%^&()]', '', new_message)
new_message = new_message.split()
X_new = np.zeros((1, len(vocab)))

for word in new_message:
    if word in vocab:
        X_new[0, list(vocab).index(word)] += 1

In [69]:
y_pred = model.predict(X_new)

In [70]:
y_pred

array([0.])

In [76]:
from sklearn.metrics import confusion_matrix

y_test_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)

[[484   1]
 [  9  64]]


In [79]:
# Accuracy
accuracy = (conf_matrix[0, 0] + conf_matrix[1, 1]) / len(y_test)

# Precision
precision = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[0, 1])

# Recall
recall = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

Accuracy: 0.982078853046595
Precision: 0.9846153846153847
Recall: 0.8767123287671232


In [84]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       485
         1.0       0.98      0.88      0.93        73

    accuracy                           0.98       558
   macro avg       0.98      0.94      0.96       558
weighted avg       0.98      0.98      0.98       558



# TEST

In [25]:
import pandas as pd
import numpy as np
from collections import Counter
import re

In [15]:
df = pd.read_csv('E:/VSCode/Python/Machine_Learning/Data/smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


## Lọc các đoạn văn bản thuộc lớp spam và ham

In [27]:
# Chuyển hết message thành chữ thường
df['message'] = df['message'].apply(lambda x: x.lower())

# Xóa các dấu câu
df['message'] = df['message'].str.replace('[.,?*!#@]', '', regex=True)

In [28]:
ham_texts = df.loc[df['label'] == 'ham', 'message']
spam_texts = df.loc[df['label'] == 'spam', 'message']

print("Ham texts:")
print(ham_texts.head())

print("\nSpam texts:")
print(spam_texts.head())

Ham texts:
0    go until jurong point crazy available only in ...
1                              ok lar joking wif u oni
3          u dun say so early hor u c already then say
4    nah i don't think he goes to usf he lives arou...
6    even my brother is not like to speak with me t...
Name: message, dtype: object

Spam texts:
2     free entry in 2 a wkly comp to win fa cup fina...
5     freemsg hey there darling it's been 3 week's n...
8     winner as a valued network customer you have b...
9     had your mobile 11 months or more u r entitled...
11    six chances to win cash from 100 to 20000 poun...
Name: message, dtype: object


## Đếm tần suất xuất hiện của mỗi từ trong mỗi nhóm

In [29]:
# Kết hợp tất cả các tin nhắn ham thành một chuỗi duy nhất
ham_combined = ' '.join(ham_texts)

# Kết hợp tất cả các tin nhắn spam thành một chuỗi duy nhất
spam_combined = ' '.join(spam_texts)

# Đếm số lần xuất hiện của từng từ trong chuỗi
ham_word_frequency = Counter(ham_combined.split())
spam_word_frequency = Counter(spam_combined.split())

print("Tần suất xuất hiện các từ trong ham:")
print(ham_word_frequency)

print("\nTần suất xuất hiện các từ trong spam:")
print(spam_word_frequency)

Tần suất xuất hiện các từ trong ham:
Counter({'i': 2187, 'you': 1834, 'to': 1562, 'the': 1126, 'a': 1059, 'u': 975, 'and': 848, 'in': 813, 'me': 756, 'my': 746, 'is': 734, 'it': 582, 'of': 524, 'for': 507, 'that': 485, 'have': 441, 'but': 424, 'your': 414, 'so': 411, 'not': 411, 'are': 410, 'on': 389, 'at': 379, 'can': 376, 'do': 376, "i'm": 372, 'if': 350, 'will': 339, 'be': 330, '2': 304, 'get': 301, 'we': 298, 'just': 290, 'now': 288, 'up': 284, '&lt;&gt;': 276, 'when': 274, 'no': 272, 'with': 271, 'ok': 267, 'how': 254, 'go': 250, 'ur': 242, 'this': 241, 'all': 240, 'what': 239, 'know': 232, 'like': 231, 'got': 230, 'call': 230, 'or': 227, 'come': 226, 'good': 224, 'then': 223, 'was': 222, 'out': 211, 'am': 211, 'its': 204, 'love': 187, 'time': 186, 'there': 184, 'he': 184, 'day': 180, '4': 174, 'ü': 169, "i'll": 168, 'going': 166, 'want': 164, 'one': 163, 'home': 160, 'lor': 160, 'about': 159, 'need': 155, 'sorry': 153, 'from': 150, 'still': 147, 'as': 146, 'see': 138, 'n': 137, '

## Tính xác suất xuất hiện của mỗi từ trong mỗi nhóm

In [30]:
# Tính tổng số từ trong nhóm ham_texts
total_ham_words = sum(ham_word_frequency.values())

# Tính xác suất xuất hiện của mỗi từ
ham_word_prob = {word: freq / total_ham_words for word, freq in ham_word_frequency.items()}

# In ra một số xác suất để kiểm tra
for word, prob in list(ham_word_prob.items()):
    print(f"Word: {word}, Probability: {prob:.6f}")

Word: go, Probability: 0.003660
Word: until, Probability: 0.000322
Word: jurong, Probability: 0.000015
Word: point, Probability: 0.000190
Word: crazy, Probability: 0.000146
Word: available, Probability: 0.000190
Word: only, Probability: 0.001889
Word: in, Probability: 0.011904
Word: bugis, Probability: 0.000102
Word: n, Probability: 0.002006
Word: great, Probability: 0.001464
Word: world, Probability: 0.000439
Word: la, Probability: 0.000102
Word: e, Probability: 0.001142
Word: buffet, Probability: 0.000029
Word: cine, Probability: 0.000102
Word: there, Probability: 0.002694
Word: got, Probability: 0.003368
Word: amore, Probability: 0.000015
Word: wat, Probability: 0.001391
Word: ok, Probability: 0.003909
Word: lar, Probability: 0.000556
Word: joking, Probability: 0.000088
Word: wif, Probability: 0.000395
Word: u, Probability: 0.014275
Word: oni, Probability: 0.000059
Word: dun, Probability: 0.000805
Word: say, Probability: 0.001332
Word: so, Probability: 0.006018
Word: early, Probabil

In [31]:
# Tính tổng số từ trong nhóm ham_texts
total_spam_words = sum(spam_word_frequency.values())

# Tính xác suất xuất hiện của mỗi từ
spam_word_prob = {word: freq / total_spam_words for word, freq in spam_word_frequency.items()}

# In ra một số xác suất để kiểm tra
for word, prob in list(spam_word_prob.items()):
    print(f"Word: {word}, Probability: {prob:.6f}")

Word: free, Probability: 0.012122
Word: entry, Probability: 0.001459
Word: in, Probability: 0.004097
Word: 2, Probability: 0.009596
Word: a, Probability: 0.021213
Word: wkly, Probability: 0.000786
Word: comp, Probability: 0.000561
Word: to, Probability: 0.038554
Word: win, Probability: 0.003255
Word: fa, Probability: 0.000224
Word: cup, Probability: 0.000281
Word: final, Probability: 0.000898
Word: tkts, Probability: 0.000224
Word: 21st, Probability: 0.000112
Word: may, Probability: 0.000393
Word: 2005, Probability: 0.000168
Word: text, Probability: 0.006678
Word: 87121, Probability: 0.000224
Word: receive, Probability: 0.001852
Word: question(std, Probability: 0.000112
Word: txt, Probability: 0.007632
Word: rate)t&c's, Probability: 0.000112
Word: apply, Probability: 0.001571
Word: 08452810075over18's, Probability: 0.000112
Word: freemsg, Probability: 0.000337
Word: hey, Probability: 0.000281
Word: there, Probability: 0.000617
Word: darling, Probability: 0.000112
Word: it's, Probabilit

## Dự đoán

In [33]:
def predict_message(message, spam_word_prob, ham_word_prob):
    words = message.lower().split()
    
    p_spam = 1.0
    s_ham = 1.0
    
    for word in words:
        p_spam *= spam_word_prob.get(word, 1e-6)
        s_ham *= ham_word_prob.get(word, 1e-6)
    
    if p_spam > s_ham:
        return 'spam'
    else:
        return 'ham'

# Example usage
message = "free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's"
message = re.sub('[.,?*!#@]', '', message)

prediction = predict_message(message, spam_word_prob, ham_word_prob)
print(f"The message is predicted to be: {prediction}")

The message is predicted to be: spam
